In [1]:
import crunch
import numpy as np
import pandas as pd
import joblib

In [ ]:
def train(X_train: pd.DataFrame, y_train: pd.DataFrame, model_directory_path: str = "resources") -> None:

    # basic xgboost regressor
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        max_depth=4,
        learning_rate=0.1,
        n_estimators=2,
        n_jobs=-1,
        colsample_bytree=0.05
    )

    # training the model
    print("training...")
    model.fit(X_train.iloc[:,2:], y_train.iloc[:,2:])

    # make sure that the train function correctly save the trained model
    # in the model_directory_path
    model_pathname = Path(model_directory_path) / "model.joblib"
    print(f"Saving model in {model_pathname}")
    joblib.dump(model, model_pathname)

In [ ]:
def infer(X_test: pd.DataFrame, model_directory_path: str = "resources") -> pd.DataFrame:

    # loading the model saved by the train function at previous iteration
    model = joblib.load(Path(model_directory_path) / "model.joblib")

    # creating the predicted label dataframe with correct dates and ids
    y_test_predicted = X_test[["date", "id"]].copy()
    y_test_predicted["value"] = model.predict(X_test.iloc[:, 2:])

    return y_test_predicted

In [ ]:
def temporal_train_test_split(X_train_loc, y_train_loc, test_size=0.2):
    unique_dates = X_train_loc.date.unique()
    split_date = unique_dates[int(len(unique_dates)*(1-test_size))]
    X_train_local = X_train_loc[X_train_loc['date'] <= split_date]
    X_test_local = X_train_loc[X_train_loc['date'] > split_date]

    y_train_local = y_train_loc[y_train_loc['date'] <= split_date]
    y_test_local = y_train_loc[y_train_loc['date'] > split_date]

    return X_train_local, X_test_local, y_train_local, y_test_local

In [ ]:
print("Splitting (X_train, y_train) in X_train_local, X_test_local, y_train_local, y_test_local")
X_train_local, X_test_local, y_train_local, y_test_local = temporal_train_test_split(
    X_train,
    y_train,
    test_size=0.2
)

In [ ]:
train(X_train_local, y_train_local)

In [ ]:
print("Inference")
y_test_local_pred = infer(X_test_local, model_directory_path="resources")
score = spearmanr(y_test_local["y"], y_test_local_pred["value"])[0] * 100
print(f"Spearman's correlation {score}")

In [ ]:
print("Remove unused data to release memory")
del X_train, y_train, X_test, X_train_local, X_test_local, y_train_local, y_test_local

In [ ]:
crunch.test(force_first_train=True, train_frequency=2)